In [2]:
%load_ext autoreload
%autoreload 2
# You can load your python module as this:
import dataproject.example

In [3]:
dataproject.example.hello_world()

'hello world'

In [5]:
pip install yahoo-finance


The following command must be run outside of the IPython shell:

    $ pip install yahoo-finance

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [6]:
install yahoo-finance

SyntaxError: invalid syntax (<ipython-input-6-3acfb7d07062>, line 1)

In [7]:
pip install yahoo-finance


The following command must be run outside of the IPython shell:

    $ pip install yahoo-finance

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/
